### Сделаем предсказание и посчитаем метрику:

Подключим Google диск

In [105]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [106]:
path = '/gdrive/MyDrive/MLBI/kurs/'

In [107]:
import pandas as pd
from sklearn.metrics import roc_auc_score,roc_curve
from urllib import request, parse
import requests
import numpy as np

# import urllib.request
import json 

Функция для формирования зароса

In [126]:
def send_json(x):
    Age, Sex, ChestPainType, RestingBP, Cholesterol, FastingBS, RestingECG, MaxHR, ExerciseAngina, Oldpeak, ST_Slope = x
    # print(x)
    body = {"Age": int(Age),
            "Sex": Sex,
            "ChestPainType": ChestPainType,
            "RestingBP": int(RestingBP),
            "Cholesterol": int(Cholesterol),
            "FastingBS": int(FastingBS),
            "RestingECG": RestingECG,
            "MaxHR": int(MaxHR),
            "ExerciseAngina": ExerciseAngina,
            "Oldpeak": Oldpeak,
            "ST_Slope": ST_Slope}
    # print(body)
    myurl = 'http://67df-34-75-54-35.ngrok.io/'+'predict'
    headers = {'content-type': 'application/json; charset=utf-8'}
    response = requests.post(myurl, json=body, headers=headers)
    return response.json()['predictions']


Загрузим данные

In [127]:
X_test = pd.read_csv(path+"X_test.csv")
y_test = pd.read_csv(path+"y_test.csv")

Сделаем запрос для 23 пациента

In [128]:
response = send_json(X_test[["Age", "Sex", "ChestPainType", "RestingBP", "Cholesterol", "FastingBS", "RestingECG", "MaxHR", "ExerciseAngina", "Oldpeak", "ST_Slope"]].iloc[23,:])

И получим ответ нашей модели

In [129]:
response

0.4202040798468818

Сделаем то же самое, но для певых 30 пациентов

In [131]:
N = 30

In [132]:
%%time
predictions = X_test[['Age', 'Sex', 'ChestPainType', 'RestingBP', 'Cholesterol', 'FastingBS', 'RestingECG', 'MaxHR', 'ExerciseAngina', 'Oldpeak', 'ST_Slope']].iloc[:N,:].apply(lambda x: send_json(x),1)

CPU times: user 158 ms, sys: 17.4 ms, total: 176 ms
Wall time: 13.1 s


Выведем предсказанные значения первых пяти пациентов

In [135]:
predictions.values[:5]

array([0.01421884, 0.60144289, 0.87360802, 0.86060294, 0.01421884])

И тестовые данные этих же самых пациентов

In [136]:
y_test.head()

,HeartDisease
0,0
1,1
2,1
3,1
4,0


Как видим наши предсказания оказались верны

Хоть и полученных результатов немного посчитаем метрику:

In [137]:
roc_auc_score(y_score=predictions.values, y_true=y_test.values[:N])

0.8555555555555555